In [261]:
'''
Last updated: February 1st, 2025

USD/SEK ("U.S. Dollar / Swedish Krona") trading strategy that focuses on the Simple Moving Average ("SMA") trend. 
20-day SMA vs 100-day SMA, over a 5-year horizon. 
Vol is taken into consideration via an adjustment of +/- 0.5* of pip value. 
'''

import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
import matplotlib
import yfinance as yf
import datetime

#Environment-specifics
initial_aum = 600000000
risk_per_trade = 600000000 * 0.01
one_lot = 100000
margin = 2.00

#Dataframes; Data
    #Timeframe:
horizon = datetime.timedelta(365*5) #5-year horizon

    #Dataframe
USDSEK_df = yf.download("USDSEK=X", datetime.date.today()-horizon,datetime.date.today())
USDSEK_Close_Price = USDSEK_df['Close']

    #Moving Averages
sma20 = USDSEK_Close_Price.rolling(20).mean()      #blue
sma100 = USDSEK_Close_Price.rolling(100).mean()    #orange

    #Calculating our ATR
highs = []
lows = []
closes = []
ticker = "USDSEK=X"
ATR_multiple = 2.00
vol_adjustment = [.95, 1.05]

highs_array = USDSEK_df['High'].astype(np.float64).values.ravel() #Had to flatten array, as .values sometimes returns a (N,1,)) array instead of an (N,)))
lows_array = USDSEK_df['Low'].astype(np.float64).values.ravel() #Could have used .ravel() or .flatten()
closes_array = USDSEK_df['Close'].astype(np.float64).values.ravel()

assert len(highs_array) == len(lows_array) == len(closes_array), "Arrays aren't the same length"

ATR = ta.ATR(highs_array, lows_array, closes_array, timeperiod = 14)

#Trading
PNLs = []
positions = {}
trade_book = []
entry = None
exit = None

#PNL Calculation
def calc_PNL(entry, exit, direction):
    percent_PNL = (((exit / entry) - 1) * (1 if direction == "BUY" else -1))
    return percent_PNL + 1

#ATR vs ATR[Periods] comparisons
    #More than, or equal to, 1 year has passed
st_ATR_larger_than_year = lambda i, ATR: (ATR[i] > np.mean(ATR[i - 365: i ])) #"st" for short-term 
st_ATR_smaller_than_year = lambda i, ATR: (ATR[i] < np.mean(ATR[i-365: i]))
    #Less than 1 year has passed
st_ATR_larger_than_period = lambda i, ATR, count_ATR: (ATR[i] > np.mean(ATR[count_ATR: i]))
st_ATR_smaller_than_period = lambda i, ATR, count_ATR: (ATR[i] < np.mean(ATR[count_ATR: i]))

#Simple Moving Average f(x)s
    #Short-term MA crossed the Long-term MA downards @t
sell = lambda i, ticker: ((sma20.loc[sma20.index[i-1], ticker]) > (sma100.loc[sma100.index[i-1], ticker])) and ((sma20.loc[sma20.index[i], ticker]) < (sma100.loc[sma100.index[i], ticker]))
    #Short-term MA crossed the Long-term MA upwards @t
buy = lambda i, ticker: ((sma20.loc[sma20.index[i-1], ticker]) < (sma100.loc[sma100.index[i-1], ticker])) and ((sma20.loc[sma20.index[i], ticker]) > (sma100.loc[sma100.index[i], ticker]))


for i in range(len(USDSEK_Close_Price)):
    if i >=14:
        if i >= 379:
            if st_ATR_larger_than_year(i, ATR): #Short-term vol is larger than the Long-Term Average, decrease position size
                if sell(i, ticker):
                    
                    #Calculate Pip Value
                    direction = "SELL"
                    pip_value = one_lot * 0.0001 * USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]

                    #Adjust Position Size due to Vol
                    position_size = risk_per_trade / ((pip_value * vol_adjustment[1])  * ATR[i])

                    #Append to Positions Dictionary
                    positions[USDSEK_Close_Price.index[i]] = {
                        "Order": "SELL",
                        "Price": USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker],
                        "Order Size": position_size,
                        "ATR": ATR[i]}

                    #Calculate the Trade's PNL
                    if entry:
                        exit = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                        trade_PNL = calc_PNL(entry, exit, direction)
                        PNLs.append(trade_PNL)
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    else:
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]

                elif buy(i, ticker):
                    direction = "BUY"
                    pip_value = one_lot * 0.0001 * USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    position_size = risk_per_trade / ((pip_value * vol_adjustment[1])  * ATR[i])
                    positions[USDSEK_Close_Price.index[i]] = {
                        "Order": "BUY",
                        "Price": USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker],
                        "Order Size": position_size,
                        "ATR": ATR[i]}                    
                    if entry:
                        exit = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                        trade_PNL = calc_PNL(entry, exit, direction)
                        PNLs.append(trade_PNL)
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    else:
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]

            elif st_ATR_smaller_than_year(i, ATR): #Short-term ATR smaller than year's average, increase position size
                if sell(i, ticker):
                    direction = "SELL"
                    pip_value = one_lot * 0.0001 * USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    position_size = risk_per_trade / ((pip_value * vol_adjustment[0])  * ATR[i])
                    positions[USDSEK_Close_Price.index[i]] = {
                        "Order": "SELL",
                        "Price": USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker],
                        "Order Size": position_size,
                        "ATR": ATR[i]}                
                    if entry:
                        exit = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                        trade_PNL = calc_PNL(entry, exit, direction)
                        PNLs.append(trade_PNL)
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    else:
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]

                elif buy(i, ticker):
                    direction = "BUY"
                    pip_value = one_lot * 0.0001 * USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    position_size = risk_per_trade / ((pip_value * vol_adjustment[0])  * ATR[i])
                    positions[USDSEK_Close_Price.index[i]] = {
                        "Order": "BUY",
                        "Price": USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker],
                        "Order Size": position_size,
                        "ATR": ATR[i]}     
                    if entry:
                        exit = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                        trade_PNL = calc_PNL(entry, exit, direction)
                        PNLs.append(trade_PNL)
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    else:
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
        elif i < 379:
            count_ATR = np.count_nonzero(ATR[14:i])
        #if ATR is bigger, at this moment, as the past 14 day ATR
            if st_ATR_larger_than_period(i, ATR, count_ATR):
                #trade smaller position due to larger vol
                if sell(i, ticker):
                    direction = "SELL"
                    #Calculate Pip Value
                    pip_value = one_lot * 0.0001 * USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]

                    #Adjust Position Size due to Vol
                    position_size = risk_per_trade / ((pip_value * vol_adjustment[1])  * ATR[i])

                    #Append to Positions Dictionary
                    positions[USDSEK_Close_Price.index[i]] = {
                        "Order": "SELL",
                        "Price": USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker],
                        "Order Size": position_size,
                        "ATR": ATR[i]}

                    #Calculate the Trade's PNL
                    if entry:
                        exit = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                        trade_PNL = calc_PNL(entry, exit, direction)
                        PNLs.append(trade_PNL)
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    else:
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]

                elif buy(i, ticker):
                    direction = "BUY"
                    pip_value = one_lot * 0.0001 * USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    position_size = risk_per_trade / ((pip_value * vol_adjustment[1])  * ATR[i])
                    positions[USDSEK_Close_Price.index[i]] = {
                        "Order": "BUY",
                        "Price": USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker],
                        "Order Size": position_size,
                        "ATR": ATR[i]}                    
                    if entry:
                        exit = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                        trade_PNL = calc_PNL(entry, exit, direction)
                        PNLs.append(trade_PNL)
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    else:
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]

            elif st_ATR_smaller_than_period(i, ATR, count_ATR):
                if sell(i, ticker):
                    direction = "SELL"
                    pip_value = one_lot * 0.0001 * USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    position_size = risk_per_trade / ((pip_value * vol_adjustment[0])  * ATR[i])
                    positions[USDSEK_Close_Price.index[i]] = {
                        "Order": "SELL",
                        "Price": USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker],
                        "Order Size": position_size,
                        "ATR": ATR[i]}                
                    if entry:
                        exit = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                        trade_PNL = calc_PNL(entry, exit, direction)
                        PNLs.append(trade_PNL)
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    else:
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]

                elif buy(i, ticker):
                    direction = "BUY"
                    pip_value = one_lot * 0.0001 * USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    position_size = risk_per_trade / ((pip_value * vol_adjustment[0])  * ATR[i])
                    positions[USDSEK_Close_Price.index[i]] = {
                        "Order": "BUY",
                        "Price": USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker],
                        "Order Size": position_size,
                        "ATR": ATR[i]}     
                    if entry:
                        exit = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                        trade_PNL = calc_PNL(entry, exit, direction)
                        PNLs.append(trade_PNL)
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]
                    else:
                        entry = USDSEK_Close_Price.loc[USDSEK_Close_Price.index[i], ticker]


[*********************100%***********************]  1 of 1 completed


In [273]:
'''
Strategy Evaluation
'''

#Sharpe Ratio
US5Y = yf.download("^FVX", datetime.date.today()) 
rf = US5Y['Close'].values.item()/100 #U.S. 5-Year Treasury ("rf")
risk_premia = ((np.cumprod(PNLs)[-1])-1) - rf
σ = np.std(np.array(PNLs))

#Max Drawdown
def max_drawdown(PNLs):
    peak = np.maximum.accumulate(np.cumprod(PNLs))
    trough = np.minimum.accumulate(np.cumprod(PNLs))
    drawdowns = (trough - peak) / peak
    return abs(np.min(drawdowns)) * 100

#Win Rate
win = 0
loss = 0
for pnl in (PNLs):
    if pnl > 1:
        win += 1
    else:
        loss += 1
win_rate = (win / (win + loss))*100

print("PROFITABILITY\n")
print(f"%PNL:{(np.cumprod(PNLs)[-1] - 1)*100: .3f}%")
print(f"$PNL:{(((np.cumprod(PNLs)[-1]) * initial_aum)-initial_aum): ,.0f}")
print(f"Sharpe Ratio:{(risk_premia/σ): .3f}x")
print(f"Max Drawdown:{max_drawdown(PNLs): .3f}%")
print(f"Win Rate:{win_rate: .3f}%")

[*********************100%***********************]  1 of 1 completed

PROFITABILITY

%PNL: 11.605%
$PNL: 69,630,487
Sharpe Ratio: 1.535x
Max Drawdown: 17.556%
Win Rate: 93.750%
